In [1]:
!pip install --quiet --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import os
import io
from openhexa.sdk import workspace
# import os.path

from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

In [3]:
conn = workspace.custom_connection('credentials-api-google-drive')

In [4]:
scopes = ["https://www.googleapis.com/auth/drive"]

creds = Credentials.from_service_account_info(eval(conn.credentials), scopes=scopes)

service = build('drive', 'v3', credentials=creds)

In [5]:
def upload_file(file_path, parent_id):
    """
    Ce programme sera principalement utilisé pour exporter le fichier généré dans un repertoire drive partagé.
    """
    mimetype ='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'

    file_name = os.path.basename(file_path)
    
    file_metadata = {
        'name': file_name.upper(),
        'parents': [parent_id]
    }
    try:
        media = MediaFileUpload(file_path, mimetype=mimetype)
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    
    except HttpError as error:
        print(f"An error occurred: {error}")
        file = None

    return file.get("id")

In [6]:
def get_share_link(file_id):
    "Permet d'obtenir le lien partagé du fichier"
    request_body={
        'role': 'reader',
        'type': 'anyone'
        }
    try:
        response_permission = service.permissions().create(fileId=file_id, body=request_body).execute()
        response_share_link = service.files().get(fileId=file_id, fields='webViewLink').execute()
    except HttpError as error:
        print(f"An error occurred: {error}")
        response_share_link = None
    
    return response_share_link.get('webViewLink')

In [8]:
def upload_file_and_get_share_link(file_path, date_report, parent_id='1LHsczOhs4f5Z4DZkuMiiYGiwj_ppKb_Z'):
    
    sub_folder_id = check_if_folder_exist(date_report, parent_id)
    # S'assurer que le fichier n'existe pas déjà dans le drive si tel est le cas alors il le supprime
    delete_file_if_exist(service, file_path)
    file_id = upload_file(file_path, sub_folder_id)
    
    return get_share_link(file_id)

In [9]:
def check_if_folder_exist(date_report, parent_id):
    import pandas as pd
    try:
        # Check if directory exist
        year = str(pd.to_datetime(date_report).year)
        query = f"name='{year}' and parents='{parent_id}' and mimeType='application/vnd.google-apps.folder'"
        response = service.files().list(q=query, fields="files(id)").execute()
        files = response.get('files', [])

        # If folder does not exist, create it
        if not files:
            file_metadata = {
                "name": year,
                "mimeType": "application/vnd.google-apps.folder",
                "parents": [parent_id]
            }
            file = service.files().create(body=file_metadata, fields="id").execute()
            return file.get('id')
        
        # Return the ID of the existing folder
        return files[0]['id']

    except HttpError as error:
        print(f"An error occurred: {error}")

In [15]:
def delete_file_if_exist(service, file_path):
    try:
        file_name = os.path.basename(file_path)
        query = f"name='{file_name}'"
        results = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
        files = results.get('files', [])

        if files:
            for file in files:
                print(f"Fichier trouvé : {file['name']} (ID: {file['id']})")
                # Supprimer le file
                service.files().delete(fileId=file['id']).execute()
                print(f"Fichier supprimé : {file['name']}")
        
        else:
            print("Aucun file trouvé avec ce nom.")
    except HttpError as error:
        print(f"Une erreur s'est produite : {error}")

In [11]:
upload_file(os.getcwd()+'/new_rapportage_feedback.xlsx', '1LHsczOhs4f5Z4DZkuMiiYGiwj_ppKb_Z')

'17iOJ6laHxq69OpezqhRMHj6MFoX3JFcz'

In [16]:
delete_file_if_exist(service, os.getcwd()+'/new_rapportage_feedback.xlsx')

new_rapportage_feedback.xlsx
Aucun file trouvé avec ce nom.


In [14]:
scopes = ["https://www.googleapis.com/auth/drive"]

creds = Credentials.from_service_account_info(eval(conn.credentials), scopes=scopes)

service = build('drive', 'v3', credentials=creds)

def upload_file(file_path, mime_type):
    file_metadata = {'name': os.path.basename(file_path)}
    media = MediaFileUpload(file_path, mimetype=mime_type)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'File ID: {file.get("id")}')


def download_file(file_id, file_path):
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(file_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f'Download {int(status.progress() * 100)}%.')

In [15]:
upload_file(os.getcwd()+'/new_rapportage_feedback.xlsx', mime_type='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

File ID: 1F11vAINDgfwbDdOrn6YeetPPzSDM0pXI


In [16]:
download_file('1F11vAINDgfwbDdOrn6YeetPPzSDM0pXI', os.getcwd()+'/new_rapportage_feedback_test.xlsx')

Download 100%.


In [17]:
file_metadata = {
    "name": "Rapport Feedback Creation",
    "mimeType": "application/vnd.google-apps.folder",
    "parents": ['1ZmViUd0IFSQzQ6Hiv_iGFLuQ2lPtdImi']
}

file = service.files().create(body=file_metadata, fields="*").execute()

print(f'Folder ID: "{file.get("id")}"')

Folder ID: "16EXAit-rbsOUH7crbHaULMlbafU4z1Lw"


In [18]:
file

{'kind': 'drive#file',
 'id': '16EXAit-rbsOUH7crbHaULMlbafU4z1Lw',
 'name': 'Rapport Feedback Creation',
 'mimeType': 'application/vnd.google-apps.folder',
 'starred': False,
 'trashed': False,
 'explicitlyTrashed': False,
 'parents': ['1ZmViUd0IFSQzQ6Hiv_iGFLuQ2lPtdImi'],
 'spaces': ['drive'],
 'version': '1',
 'webViewLink': 'https://drive.google.com/drive/folders/16EXAit-rbsOUH7crbHaULMlbafU4z1Lw',
 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/vnd.google-apps.folder+48+shared',
 'hasThumbnail': False,
 'thumbnailVersion': '0',
 'viewedByMe': False,
 'createdTime': '2024-10-16T09:17:31.016Z',
 'modifiedTime': '2024-10-16T09:17:31.016Z',
 'modifiedByMeTime': '2024-10-16T09:17:31.016Z',
 'modifiedByMe': True,
 'owners': [{'kind': 'drive#user',
   'displayName': 'automatingdaptools@electric-facet-438520-h7.iam.gserviceaccount.com',
   'photoLink': 'https://lh3.googleusercontent.com/a/ACg8ocLpcq5tEr1kdzuR9IC-17bk3HQ-bPmociNjvHfEkp_5d3ghLU4=s64',
   'me'

In [20]:
service_file = service.files()
dir(service_file)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_credentials_validated',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 '_universe_domain',
 '_validate_credentials',
 'close',
 'copy',
 'create',
 'delete',
 'download',
 'emptyTrash',
 'export',
 'export_media',
 'generateIds',
 'get',
 'get_media',
 'list',
 'listLabels',
 'listLabels_next',
 'list_next',
 'modifyLabels',
 'update',
 'watch']

In [21]:
# Liste tous les dossiers présent dans un dossier spécifique

import pandas as pd

query = f"parents ='1ZmViUd0IFSQzQ6Hiv_iGFLuQ2lPtdImi'"
response = service_file.list(q=query).execute()
files = response.get('files')
nextPageToken = response.get('nextPageTOken')

while nextPageToken:
    response = service_file.files().list(q=query).execute()
    files.extend(response.get('files'))
    nextPageToken = response.get('nextPageTOken')

df = pd.DataFrame(files)

df

,kind,mimeType,id,name
0,drive#file,application/vnd.google-apps.folder,16EXAit-rbsOUH7crbHaULMlbafU4z1Lw,Rapport Feedback Creation


In [ ]:
# Permission sharing file type

file_id =''

request_body={
    'role': 'reader',
    'type': 'anyone'
}

response_permission = service.permissions().create(
    fileId=file_id,
    body=request_body
).execute


response_share_link = service.files().get(
    fileId=file_id,
    fields='webViewLink'
).execute

In [41]:
mimetype ={
    '.xlsx': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    }

mimetype.get(os.path.splitext(file_path)[-1])

'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'

In [27]:
import sys

path = '/home/jovyan/workspace/automating_dap_tools/code/feedback_report/'

if path not in sys.path:
    sys.path.append(path)

from importlib import reload

from pipeline_feeedback_report.export_file_to_google_drive import upload_file_to_drive as drive

reload(drive)

<module 'pipeline_feeedback_report.export_file_to_google_drive.upload_file_to_drive' from '/home/jovyan/workspace/automating_dap_tools/code/feedback_report/pipeline_feeedback_report/export_file_to_google_drive/upload_file_to_drive.py'>

In [28]:
drive.upload_file_and_get_share_link(os.getcwd()+'/new_rapportage_feedback.xlsx', '31-03-2024')

'https://docs.google.com/spreadsheets/d/1Ct-Sa7rypdB8dtgwAX7fklomBFoConCq/edit?usp=drivesdk&ouid=104898089442189668678&rtpof=true&sd=true'

In [ ]:
https://docs.google.com/spreadsheets/d/1mrXNNkStLehuzGMVe_sjtXtPYo4s6786/uc?usp=drivesdk&ouid=104898089442189668678&rtpof=true&sd=true&export=download

In [32]:
os.path.abspath('credentials.json')

'/home/jovyan/workspace/automating_dap_tools/code/feedback_report/pipeline_feeedback_report/export_file_to_google_drive/credentials.json'

In [35]:
os.path.join(os.getcwd()+'/export_file_to_google_drive', 'credentials.json')

'/home/jovyan/workspace/automating_dap_tools/code/feedback_report/pipeline_feeedback_report/export_file_to_google_drive/export_file_to_google_drive/credentials.json'